In [66]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json
import requests

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token


In [67]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [59]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]

In [68]:
def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    if result.status_code ==200:
        json_result=json.loads(result.content)
        return json_result['tracks']
    else:
        print(f"Failed to fetch recommendations. Status code: {result.status_code}")
        return None



In [69]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    json_result=json.loads(result.content)["tracks"]
    return json_result

In [78]:
token = get_token()
print(token)

BQDhlSAoqQAXE1r1mgmvR_oyZChKJAMf9IOJzNgGExgGSYnGyYgBe86zHtDBg2uFCCOBWPHvI0poSCxXWCVxdpwMGCx5ItM6h4smo9V1XpLG0BJWQyA


In [12]:
import pandas as pd
artist_df = pd.read_csv('top_artist.csv', index_col= 0)

In [13]:
artist_df

,Artist,Streams,Daily,As lead,Solo,As feature
0,Drake,"84,660.7",49.796,"56,989.7","32,551.1","27,671.0"
1,Bad Bunny,"67,185.3",45.347,"40,770.4","22,952.6","26,414.8"
2,Taylor Swift,"57,136.6",97.277,"54,855.2","49,763.0","2,281.5"
3,The Weeknd,"53,312.6",45.820,"42,401.2","30,997.4","10,911.4"
4,Ed Sheeran,"47,777.7",17.885,"42,658.7","33,820.8","5,118.9"
...,...,...,...,...,...,...
2995,The Mayries,724.3,0.124,723.4,697.4,0.9
2996,slowthai,724.3,0.229,504.8,271.9,219.4
2997,Charles Aznavour,723.1,0.419,718.7,639.3,4.3
2998,Jok'air,722.8,0.385,582.0,363.2,140.8


In [39]:
artists = artist_df['Artist']

In [38]:
artist_id_list = []
for artist in artists:
    artist_id = search_for_artist(token, artist)
    artist_id_list.append(artist_id['id'])

artist_id_list

KeyError: 'artists'

In [29]:
artist_dict = {
    'artist':artists,
    'artist_id':artist_id_list
}
artist_id_df = pd.DataFrame(artist_dict)
artist_id_df

,artist,artist_id
0,Drake,3TVXtAsR1Inumwj472S9r4
1,Bad Bunny,4q3ewBCX7sLwd24euuV69X
2,Taylor Swift,06HL4z0CvFAxyc27GXpf02
3,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ
4,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V
...,...,...
2995,The Mayries,38SWPOPO1YqxUPnT4AAoID
2996,slowthai,3r1XkJ7vCs8kHBSzGvPLdP
2997,Charles Aznavour,2hgP9Ap2tc10R5jrQaEpMT
2998,Jok'air,2kIs76sEGiulKeqetZq6ua


In [31]:
artist_id_df.to_csv('artist_id.csv')

In [ ]:
artist_id_df = pd.read_csv('artist_id.csv')

In [43]:
artist_id_list = artist_id_df['artist_id']

In [76]:
artist_chunk1 = artist_id_list[0:1000]
artist_chunk2 = artist_id_list[1000:2000]
artist_chunk3 = artist_id_list[2000:3000]

In [83]:

recommended_song_list1 = []
recommended_song_id1 = []

for artist_id in artist_chunk1:
    song_low =  get_songs_recommendations(token, artist_id, 0, 33)
    song_med =  get_songs_recommendations(token, artist_id, 34, 66)
    song_high =  get_songs_recommendations(token, artist_id, 67, 100)
    songs = []
    songs.extend(song_low)
    songs.extend(song_med)
    songs.extend(song_high)
    for song in songs:
        # print(song['name'])
        # print(song['id'])
        recommended_song_id1.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity'],

            }
            recommended_song_list1.append(dict)


In [84]:
token = get_token()
print(token)

BQBBxZGZGyVB-aOF2FZIUroLs2MeKd93gG3gP3IOcWKLfDGArrxfFidIIeCjIoS1H9ZqfQX5EbUOtk-gmsJQ7GZJdBXn0gH1p1bsZ2_HiQ1AfoLTFeQ


In [85]:
recommended_song_list2 = []
recommended_song_id2 = []

for artist_id in artist_chunk2:
    song_low =  get_songs_recommendations(token, artist_id, 0, 33)
    song_med =  get_songs_recommendations(token, artist_id, 34, 66)
    song_high =  get_songs_recommendations(token, artist_id, 67, 100)
    songs = []
    songs.extend(song_low)
    songs.extend(song_med)
    songs.extend(song_high)
    for song in songs:
        # print(song['name'])
        # print(song['id'])
        recommended_song_id2.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity'],

            }
            recommended_song_list2.append(dict)


In [91]:
token = get_token()
print(token)

BQAud02_FaXDB0ImT6mEg8-oYFpqqoOImK-yc-Rp_a1aGgVCUh19PYFXj6W1nZS5R-gQJyReNQ2fTZksw1PIc91pc6Y-CsqQjV8oY5jk-uzyuTXdeyo


In [87]:
recommended_song_list3 = []
recommended_song_id3 = []


for artist_id in artist_chunk3:
    song_low =  get_songs_recommendations(token, artist_id, 0, 33)
    song_med =  get_songs_recommendations(token, artist_id, 34, 66)
    song_high =  get_songs_recommendations(token, artist_id, 67, 100)
    songs = []
    songs.extend(song_low)
    songs.extend(song_med)
    songs.extend(song_high)
    for song in songs:
        # print(song['name'])
        # print(song['id'])
        recommended_song_id3.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity'],

            }
            recommended_song_list3.append(dict)


In [88]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [89]:
def get_song_audio_features(token, song_id):
    url = f"https://api.spotify.com/v1/audio-features?ids={','.join(song_id)}"
    headers = get_auth_header(token)
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        json_result = response.json()  # Use .json() method to parse the JSON data
        return json_result
    else:
        print(f"Failed to fetch audio features. Status code: {response.status_code}")
        return None


In [92]:
recommended_song_id = []
recommended_song_id.extend(recommended_song_id1)
recommended_song_id.extend(recommended_song_id2)
recommended_song_id.extend(recommended_song_id3)

In [94]:
recommended_features = []
for group in chunker(recommended_song_id, 100):
    recommended_feature_group = get_song_audio_features(token, group)
    recommended_features.append(recommended_feature_group)
    

In [95]:
recommended_song_list = []
recommended_song_list.extend(recommended_song_list1)
recommended_song_list.extend(recommended_song_list2)
recommended_song_list.extend(recommended_song_list3)

In [96]:
import pandas as pd

In [100]:
df_recommended_songs = pd.DataFrame(recommended_song_list)


In [102]:
df_recommended_songs = df_recommended_songs.drop_duplicates(subset = ['name']).reset_index(drop = True)
df_recommended_songs

,name,artist,id,popularity
0,"Chanel Vintage (feat. Future, Young Thug)",Metro Boomin,6y7mf352StjAm4bMDkeMyH,31
1,Slow It Down,Short Dawg,6GgjPqWh25IVIPNTp1soPd,32
2,Dedicated,The Game,4HLElWheO0EfBtWQFkeOcG,31
3,Dont Worry (its on me) - Radio Edit,Sir William,0FcjlhzxLTUnNseW0aNjrg,24
4,Club Paradise - Remix,Kigity K,30XUZHuO3Zn79Kpe5ylkXI,21
...,...,...,...,...
238099,Room for Space,Jaxxon D. Silva,4jQcMPX9r6PwfFayqqs20l,35
238100,hero,juno,4F8FtOWSa286XpVEeB8pWm,53
238101,Time Won't Stop,Yung Cortex,6sxWt3iBJijlPVb6aENIYR,44
238102,STOP DROP & ROLL,NotNevi,0whJqH0WD0R4gsRA2TznM6,40


In [ ]:
df_recommended_audiofeatures = pd.DataFrame(recommended_features['audio_features'])
df_recommended_audiofeatures

,audio_features
0,"[{'danceability': 0.823, 'energy': 0.704, 'key..."
1,"[{'danceability': 0.519, 'energy': 0.852, 'key..."
2,"[{'danceability': 0.866, 'energy': 0.613, 'key..."
3,"[{'danceability': 0.886, 'energy': 0.57, 'key'..."
4,"[{'danceability': 0.72, 'energy': 0.63, 'key':..."
...,...
6688,"[{'danceability': 0.605, 'energy': 0.22, 'key'..."
6689,"[{'danceability': 0.809, 'energy': 0.782, 'key..."
6690,"[{'danceability': 0.882, 'energy': 0.675, 'key..."
6691,"[{'danceability': 0.611, 'energy': 0.854, 'key..."
